In [163]:
from kafka import KafkaConsumer
from time import sleep
import json
import re
import pandas as pd
import tqdm

##  Créer un topic nommé test

In [164]:
from confluent_kafka.admin import AdminClient, NewTopic


admin_client = AdminClient({
    "bootstrap.servers": "localhost:9092"
})

topic_list = []
topic_list.append(NewTopic("test", 1, 1))
admin_client.create_topics(topic_list)

{'test': <Future at 0x1f51c1ea2c8 state=running>}

## Créer un consommateur qui va recevoir le traffic depuis le topic test

In [165]:
consumer = KafkaConsumer('test',bootstrap_servers=['localhost:9092'],api_version=(0,10))

## Préparation de données

In [166]:
# There is a minor bug in this regex, it misses the last field. I'll fix this soon. 

common_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-)'
combined_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<useragent>[^"]*)'
columns = ['client', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'user_agent']

In [167]:
# Cette fonction permet de transformer une ligne en un dictionnaire bien structuré pour faciliter le stockage dans mongoDB
def get_dic(line):
    dic={'client':line[0],'datetime':pd.to_datetime(line[2],format='%d/%b/%Y:%H:%M:%S %z'),'method':line[3],'request':line[4],'status':line[5],'size':line[6],'referer':line[7],'user_agent':line[8]}
    return dic

In [168]:
# Connection à la base de données logs_db
import pymongo
def connect_db():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["logs_db"]
    mycoll = mydb["logs"]
    return mycoll

In [169]:
# Insérer un document dans la base de données
def insert_line(line,mycol):
    try:
        log_line = re.findall(combined_regex, line)[0]
        dic = get_dic(log_line)
        mycol.insert_one(dic)
    except Exception as e:
        print()
     
        

In [170]:
mycol = connect_db()

In [ ]:
# Stocker les documents traités dans la base de données
for message in consumer:
    line = json.loads(message.value)
    insert_line(line,mycol)


In [77]:
#dic = {"nom":"ismail","age":23}
#mycol.insert_one(dic)
